# DKN : Deep Knowledge-Aware Network for News Recommendation

## Global settings and imports

In [ ]:
import os
import sys
import logging
from pathlib import Path
import zipfile
from time import time
import polars as pl

import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

logging.basicConfig(level=logging.INFO)
LOG = logging.getLogger(__name__)
LOG.setLevel(logging.INFO)

from recommenders.datasets.download_utils import maybe_download
from recommenders.models.deeprec.deeprec_utils import prepare_hparams
from recommenders.models.deeprec.models.dkn import DKN
from recommenders.models.deeprec.io.dkn_iterator import DKNTextIterator

from group_33.util import train_test_split
from group_33.dkn import transform_behaviors, transform_history, tokenize_articles, create_feature_file, transform_behaviors_test, calculate_rankings

print(f"System version: {sys.version}")
print(f"Tensorflow version: {tf.__version__}")

In [ ]:
# DKN parameters
epochs = 10
history_size = 50
batch_size = 1000

DATASET_NAME = "small" # one of: demo, small, large
GROUP_PATH = Path("../shared/194.035-2024S/groups/Gruppe_33/Group_33")


# prepare tmp dir
tmp_path = GROUP_PATH / "tmp" / "dkn" #Path("..", "tmp", "dkn")
tmp_data_path = tmp_path / DATASET_NAME
(tmp_data_path / "validation").mkdir(exist_ok=True, parents=True)
(tmp_data_path / "train").mkdir(exist_ok=True, parents=True)
(tmp_data_path / "evaluation").mkdir(exist_ok=True, parents=True)

tmp_test_path = tmp_path / "test"
tmp_test_path.mkdir(exist_ok=True, parents=True)

# train & validation & evaluation
data_path = GROUP_PATH / "data" / DATASET_NAME
train_file = tmp_data_path / "train" / "behaviours.txt"
valid_file = tmp_data_path / "validation" / "behaviours.txt"
evaluation_file = tmp_data_path / "evaluation" / "behaviors.txt"
user_history_file = tmp_data_path / "user_history.txt"
articles_file = data_path / "articles.parquet"
articles_tokenized_file = tmp_data_path / "articles_tokenized.parquet"
word_embeddings_file = tmp_data_path / "word_embeddings.npy"
entity_embeddings_file = tmp_data_path / "entity_embeddings.npy"
context_embeddings_file = tmp_data_path / "context_embeddings.npy"
news_feature_file = tmp_data_path / "news_feature.txt"
infer_embedding_file = tmp_data_path / "infer_embedding.txt"

# test
test_raw_file = data_path / ".." / "ebnerd_testset" / "test" / "behaviors.parquet"
test_file = tmp_test_path / "behavior.txt"
test_articles_file = GROUP_PATH / "data" / "test" / "articles.parquet"
test_articles_tokenized_file = tmp_test_path / "articles_tokenized.parquet"

# prediction
indexed_behaviors_file = tmp_data_path / "indexed_behaviors.parquet"
scores_file = tmp_data_path / "scores.txt"
predictions_file = tmp_data_path / "predictions.txt"

LOG.info(data_path)
LOG.info(tmp_path)

pl.Config.set_tbl_rows(100)

run_train = True if os.environ.get("TRAIN") else False
print(run_train)

## Data preparation

In [ ]:
if not (tmp_path / "model.bin").exists():
    maybe_download("http://vectors.nlpl.eu/repository/20/38.zip", tmp_path / "word2vec.zip")

    with zipfile.ZipFile(tmp_path / "word2vec.zip", 'r') as zip_ref:
        zip_ref.extractall(tmp_path)

In [ ]:
from group_33.util import train_test_split
pl.Config.set_streaming_chunk_size(500_000)
force_reload = False

if not train_file.exists() or force_reload:
    train = transform_behaviors(pl.scan_parquet(data_path / 'train' / 'behaviors.parquet'))
    train.sink_csv(train_file, separator=' ', quote_style='never', include_header=False)
    # train_test.collect(streaming=True).write_csv(valid_file, separator=' ', quote_style='never', include_header=False)

if not evaluation_file.exists() or force_reload:
    validation_behaviors = pl.scan_parquet(data_path / 'validation' / 'behaviors.parquet')
    validation, evaluation = train_test_split(validation_behaviors, 0.5)

    validation_transformed = transform_behaviors(validation)
    validation_transformed.collect(streaming=True).write_csv(evaluation_file, separator=' ', quote_style='never', include_header=False)

    evaluation_transformed = transform_behaviors(evaluation)
    evaluation_transformed.collect(streaming=True).write_csv(valid_file, separator=' ', quote_style='never', include_header=False)

if not user_history_file.exists() or force_reload:
    user_history = transform_history(
        data_path / 'train' / 'history.parquet',
        data_path / 'validation' / 'history.parquet',
        data_path / '..' / 'ebnerd_testset' / 'test' / 'history.parquet'
    )
    user_history.sink_csv(user_history_file, separator=' ', quote_style='never', include_header=False)

if not articles_tokenized_file.exists() or force_reload:
    tokenize_articles(articles_file, articles_tokenized_file)

if not test_articles_tokenized_file.exists() or force_reload:
    tokenize_articles(test_articles_file, test_articles_tokenized_file)

if not news_feature_file.exists() or force_reload:
    create_feature_file(
        tmp_path / "model.bin",
        articles_tokenized_file, test_articles_tokenized_file,
        word_embeddings_file, entity_embeddings_file,
        context_embeddings_file, news_feature_file, 10
    )

## Create hyper-parameters

In [ ]:
yaml_file = maybe_download(url="https://recodatasets.z20.web.core.windows.net/deeprec/deeprec/dkn/dkn_MINDsmall.yaml",
                           work_directory=data_path)
hparams = prepare_hparams(yaml_file,
                          seed=33,
                          show_step=100,
                          news_feature_file=news_feature_file.as_posix(),
                          user_history_file=user_history_file.as_posix(),
                          wordEmb_file=word_embeddings_file.as_posix(),
                          entityEmb_file=entity_embeddings_file.as_posix(),
                          contextEmb_file=context_embeddings_file.as_posix(),
                          epochs=epochs,
                          save_model=True,
                          MODEL_DIR=(tmp_path / "model" / f"{int(time())}_e{epochs}_h{history_size}").as_posix(),
                          history_size=history_size,
                          batch_size=batch_size)

## Train the DKN model

In [ ]:
model = DKN(hparams, DKNTextIterator)

In [ ]:
if run_train:
    model.fit(train_file, valid_file)
else:
    model.load_model(str( GROUP_PATH / "tmp/model/epoch_10"))

## Evaluate the DKN model

In [ ]:
res = model.run_eval(str(evaluation_file))
print(res)

## Predict for RecSys Challenge Testdata

In [ ]:
if not Path(test_file).exists():
    transform_behaviors_test(str(test_raw_file), indexed_behaviors_file, test_file)

In [ ]:
model.predict(str(test_file), scores_file)

In [ ]:
rankings = calculate_rankings(indexed_behaviors_file, scores_file)
rankings.write_csv(predictions_file, separator=" ", include_header=False)
print(f"Created predictions file at {predictions_file}")